In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from typing import Dict
import json
import os
import requests
import boto3

# 1. DONE - get one day's taxi data
# 2. DONE - get one day's weather data
# 3. DONE - upload to S3 (raw_data/to_processed/weather_data and raw_data/to_processed/taxi_data)
# 4. DONE - create functions - organize the code (docstrings and type hints)
# 5. DONE - create a trigger

def get_taxi_data(formatted_datetime: str) -> Dict:
    """ 
    Retrieves taxi data for the given date.
    
    Parameters:
        formatted_datetime (str): The date in "YYYY-MM-DD" format.
        
    Returns:
        Dict: A dictionary containing the taxi_data as a JSON.
    """
    
    taxi_url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
    headers = {'X-App-Token': os.environ.get('CHICAGO_KEY')}
    
    response_taxi = requests.get(taxi_url, headers=headers)
    taxi_data = response_taxi.json()
    
    return taxi_data
    
def get_weather_data(formatted_datetime: str) -> Dict:
    
    """
    Retrieves historical weather data for Chicago on a specified date.

    Parameters:
        formatted_datetime (str): The date for which to retrieve weather data, in "YYYY-MM-DD" format.

    Returns:
        Dict: A dictionary containing the weather data as a JSON object.
    """
    
    weather_url = 'https://archive-api.open-meteo.com/v1/era5'
    
    params = {
        'latitude': 41.85,
        'longitude': -87.65,
        'start_date': formatted_datetime,
        'end_date': formatted_datetime,
        'hourly': 'temperature_2m,wind_speed_10m,rain,precipitation'
    }

    response_weather = requests.get(weather_url, params=params)
    weather_data = response_weather.json()
    
    return weather_data
    
def upload_to_s3(data: Dict, folder_name: str, filename: str) -> None:
    
    """
    Uploads a JSON-serialized dictionary to an S3 bucket.

    Parameters:
        data (Dict): The dictionary to be uploaded.
        folder_name (str): The name of the folder within the 'raw_data/to_processed' directory in the S3 bucket.
        filename (str): The name of the file to be saved in the specified folder.

    Returns:
        None
    """
    
    client = boto3.client('s3')
    client.put_object(
        Bucket='cubix-chicago-taxi-ms', 
        Key=f'raw_data/to_processed/{folder_name}/{filename}', 
        Body=json.dumps(data)
    )

def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=8)
    formatted_datetime = current_datetime.strftime('%Y-%m-%d')
    
    taxi_data_api_call = get_taxi_data(formatted_datetime)
    weather_data_api_call = get_weather_data(formatted_datetime)
    
    taxi_filename = f'taxi_raw_{formatted_datetime}.json'
    weather_filename = f'weather_raw_{formatted_datetime}.json'
    
    upload_to_s3(data=taxi_data_api_call, filename=taxi_filename, folder_name='taxi_data')
    print('Taxi data has been uploaded!')
    
    upload_to_s3(data=weather_data_api_call, filename=weather_filename, folder_name='weather_data')
    print('Weather data has been uploaded!')